In [ ]:
using  Gmsh: gmsh
using  GridapGmsh
using  Gridap
using  Gridap.Geometry
using  Gridap.TensorValues
using  PyPlot

In [ ]:
const  E_mat = 25.8423e3
const  ν_mat = 0.18
const  Gc = 0.089
const  η = 1e-15

In [ ]:
const  ls = 1.1875
const  L = 500
const  H = 500
const  LOf = 220
const  hfc = ls/5 #Mesh  size  parameter
const  hf = ls/2.1 #Mesh  size  parameter
const  h = 40*hf #Mesh  size  parameter
const  thick = 100

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h ,1)
gmsh.model.geo.addPoint(L/2, 0.0, 0.0, h, 2)
gmsh.model.geo.addPoint(L/2, H/2, 0.0, hfc, 3)
gmsh.model.geo.addPoint(L/2+LOf-5, H/2, 0.0, hfc, 4)
gmsh.model.geo.addPoint(L/2+LOf+5, H/2, 0.0, hfc, 5)
gmsh.model.geo.addPoint(L, H/2, 0.0, h, 6)
gmsh.model.geo.addPoint(L, H, 0.0, h, 7)
gmsh.model.geo.addPoint(0.0, H, 0.0, h, 8)
gmsh.model.geo.addPoint(0.0, H/2, 0.0, hf, 9)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 9, 8)
gmsh.model.geo.addLine(9, 1, 9)
gmsh.model.geo.addLine(3, 9, 100)

gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7,8,9],1)
gmsh.model.geo.addPlaneSurface([1], 1)

gmsh.model.addPhysicalGroup(2, [1],1)
gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [4],2)

gmsh.model.setPhysicalName(2, 1, "Domain")
gmsh.model.setPhysicalName(1, 1, "Support")
gmsh.model.setPhysicalName(1, 2, "LoadLine")

gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [100])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", hf)
gmsh.model.mesh.field.setNumber(2, "LcMax", h)
gmsh.model.mesh.field.setNumber(2, "DistMin", 25*ls)
gmsh.model.mesh.field.setNumber(2, "DistMax", 35*ls)
gmsh.model.mesh.field.setAsBackgroundMesh(2)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("LShapedPanel.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("LShapedPanel.msh")
writevtk(model ,"LShapedPanel")

In [ ]:
function  ElasFourthOrderConstTensor(E,ν,PlanarState)
    # 1 for  Plane  Stress  and 2 Plane  Strain  Condition
    if  PlanarState  == 1
        C1111 =E/(1-ν*ν)
        C1122 = (ν*E)/(1-ν*ν)
        C1112 = 0.0
        C2222 =E/(1-ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    elseif  PlanarState  == 2
        C1111 = (E*(1-ν*ν))/((1+ν)*(1-ν-2*ν*ν))
        C1122 = (ν*E)/(1-ν-2*ν*ν)
        C1112 = 0.0
        C2222 = (E*(1-ν))/(1-ν-2*ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    end
    C_ten = SymFourthOrderTensorValue(C1111 ,C1112 ,C1122 ,C1112 ,C1212 ,C2212 ,C1122 ,C2212 ,C2222)
    return   C_ten
end

In [ ]:
const  C_mat = ElasFourthOrderConstTensor(E_mat ,ν_mat ,2)

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
P_vol = (1.0/3)*I4
P_dev = I4_sym  - P_vol

In [ ]:
function σfun(ε,ε_in,s_in)
    σ_elas = C_mat⊙ε
    if tr(ε_in) >= 0
        σ = (s_in ^2+η)*σ_elas
    elseif  tr(ε_in) < 0
        σ = (s_in ^2+η)*P_dev ⊙ σ_elas + P_vol ⊙ σ_elas
    end
    return  σ
end

In [ ]:
function ψPos(ε_in)
   σ_elas = C_mat⊙ε_in
    if tr(ε_in) >= 0
        ψPlus = 0.5*(ε_in ⊙ σ_elas)
    elseif  tr(ε_in) < 0
        ψPlus = 0.5*((P_dev ⊙ σ_elas)⊙(P_dev⊙ε_in))
    end
    return ψPlus
end

In [ ]:
function  new_EnergyState(ψPlusPrev_in,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in  >=ψPlusPrev_in
        ψPlus_out =ψPlus_in
    else
        ψPlus_out=ψPlusPrev_in
    end
    true,ψPlus_out
end

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = solve(op)
    return  qh
end

In [ ]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian ,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model,reffe_Disp;conformity =:H1,
    dirichlet_tags = ["Support","LoadLine"],
    dirichlet_masks =[(true ,true), (false ,true)])

uh = zero(V0_Disp)

In [ ]:
degree = 2*order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree)

In [ ]:
labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels ,"Support")
Γ_Load = BoundaryTriangulation(model ,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load ,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
function   stepPhaseField(uh_in ,ΨPlusPrev_in)
    a_PF(s,φ) =∫( Gc*ls*∇(φ)⋅∇(s)+ 2*ΨPlusPrev_in*s*φ+ (Gc/ls)*s*φ)*dΩ
    b_PF(φ) =∫( (Gc/ls)*φ)*dΩ
    op_PF = AffineFEOperator(a_PF,b_PF ,U_PF ,V0_PF)
    sh_out = solve(op_PF)
    return  sh_out
end

In [ ]:
function  stepDisp(uh_in,sh_in ,vApp)
    uApp1(x) = VectorValue(0.0,0.0)
    uApp2(x) = VectorValue(0.0,vApp)
    U_Disp = TrialFESpace(V0_Disp ,[uApp1 ,uApp2])
    a_Disp(u,v) =∫( (ε(v)⊙(σfun∘(ε(u),ε(uh_in),sh_in)) ) )*dΩ
    b_Disp(v) = 0.0
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,U_Disp ,V0_Disp)
    uh_out = solve(op_Disp)
    return  uh_out
end

In [ ]:
vApp = 0
delv = 1e-3
const  vAppMax = 0.7
innerMax = 10
count = 0
Load = Float64[]
Displacement = Float64[]
COUNTArray = Float64[]

push!(Load, 0.0)
push!(Displacement, 0.0)
sPrev = CellState(1.0,dΩ)
sh = project(sPrev ,model ,dΩ,order)
ψPlusPrev = CellState(0.0,dΩ)
while  vApp .< vAppMax
    count = count  .+ 1
    if count <= 300
        vApp = vApp+delv
    elseif count <= 600
        vApp = vApp-delv
    elseif count > 600
        vApp = vApp+delv
    end
    
    print("\n Entering  displacemtent  step$count :", float(vApp))
    for  inner = 1: innerMax
        ψhPlusPrev = project(ψPlusPrev ,model ,dΩ,order)
        RelErr = abs(sum(∫( Gc*ls*∇(sh)⋅∇(sh) + 2*ψhPlusPrev*sh*sh + (Gc/ls)*sh*sh)*dΩ-∫( (Gc/ls)*sh)*dΩ))/abs(sum(∫( (Gc/ls)*sh)*dΩ))
        print("\n Relative Error :", float(RelErr))
        sh = stepPhaseField(uh,ψhPlusPrev)
        uh = stepDisp(uh,sh,vApp)
        ψhPos_in =ψPos∘(ε(uh))
        update_state!( new_EnergyState ,ψPlusPrev ,ψhPos_in)
        if   RelErr  < 1e-8
            break
        end
    end
    Node_Force = sum(∫(n_Γ_Load⋅(σfun∘(ε(uh),ε(uh),sh)))*dΓ_Load)
    push!(Load , -thick*Node_Force[2])
    push!( Displacement, vApp)
    
    push!(COUNTArray,count)
    
    if mod(count,20) == 0
    writevtk(Ω,"results_LShapedPanel$count",cellfields= ["uh"=>uh ,"s"=>sh])
    end
end

In [ ]:
plot(Displacement,Load*1e-3)